In [1]:
import requests, json
from pathlib import Path

# USDA API Key
API_KEY = "GA0LAyk7zcLgEjKMdSfOIOl7GJmL4wRleIlflcfp"

# Base URL
BASE_URL = "https://api.fas.usda.gov"

# Parameters for requests
params = {"api_key": API_KEY}
commodity_code = "401"
market_year = "2026"
ENDPOINT = f"/api/esr/exports/commodityCode/{commodity_code}/allCountries/marketYear/{market_year}"

response = requests.get(BASE_URL + ENDPOINT, params=params)

if response.status_code == 200:
    data = response.json()
    raw_path = Path("../data/raw/corn_exports_2025.json")

    # Write JSON string to file
    file = open(raw_path, "w")
    json.dump(data, file, indent=2)
    file.close()
    print("Saved 2025 corn export data to ", raw_path)
else:
    print(f"Error: {response.status_code}, {response.text}")

Saved 2025 corn export data to  ../data/raw/corn_exports_2025.json


In [7]:
import pandas as pd

ENDPOINT = "/api/esr/datareleasedates"

response = requests.get(BASE_URL + ENDPOINT, params=params)
response.raise_for_status()

data = pd.DataFrame(response.json())
data.head(100)


,commodityCode,marketYearStart,marketYearEnd,marketYear,releaseTimeStamp
0,101,2025-06-01T00:00:00,2026-05-31T00:00:00,2026,2025-12-11T00:00:00
1,102,2025-06-01T00:00:00,2026-05-31T00:00:00,2026,2025-12-11T00:00:00
2,103,2025-06-01T00:00:00,2026-05-31T00:00:00,2026,2025-12-11T00:00:00
3,104,2025-06-01T00:00:00,2026-05-31T00:00:00,2026,2025-12-11T00:00:00
4,105,2025-06-01T00:00:00,2026-05-31T00:00:00,2026,2025-12-11T00:00:00
...,...,...,...,...,...
95,1403,2023-08-01T00:00:00,2024-07-31T00:00:00,2024,2024-12-12T00:00:00
96,1404,2023-08-01T00:00:00,2024-07-31T00:00:00,2024,2024-12-12T00:00:00
97,1498,2023-08-01T00:00:00,2024-07-31T00:00:00,2024,2024-12-12T00:00:00
98,1499,2023-08-01T00:00:00,2024-07-31T00:00:00,2024,2024-12-12T00:00:00


In [11]:
import requests
import pandas as pd

# 1. Configuration
API_KEY = 'inXQmuWzrfGPG2kWs1pjEWJuaxvgm9LzZhi7f7tg'
BASE_URL = 'https://api.eia.gov/v2/'

# Use the 'petroleum/sum/sndw/data' route for weekly summary stats
endpoint = "petroleum/sum/sndw/data"

# 2. Define the Request Parameters for U.S. TOTAL ETHANOL
params = {
    'api_key': API_KEY,
    'frequency': 'weekly',
    'data[0]': 'value',
    
    # PRODUCT: Change EPP0 to EPOOXE for Fuel Ethanol
    'facets[product][]': 'EPOOXE',
    
    # AREA: Filter for 'NUS' to get the U.S. Total and exclude PADDs
    # We include 'NUS-Z00' as well to catch all "Total" series (like Exports/Imports)
    'facets[duoarea][]': ['NUS', 'NUS-Z00'],
    
    'sort[0][column]': 'period',
    'sort[0][direction]': 'desc',
    'length': 100
}

def fetch_ethanol_totals():
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status()
        
        raw_data = response.json()
        data_list = raw_data['response']['data']
        
        df = pd.DataFrame(data_list)
        
        # Keep only the relevant columns for a clean view
        cols_to_show = ['period', 'area-name', 'product-name', 'process-name', 'series-description', 'value', 'units']
        return df[cols_to_show]

    except Exception as e:
        print(f"Error: {e}")
        return None

# Execute and display the most recent data
ethanol_df = fetch_ethanol_totals()
if ethanol_df is not None:
    print(f"Showing most recent U.S. Total Ethanol records:")
    display(ethanol_df.head(10))

Showing most recent U.S. Total Ethanol records:


,period,area-name,product-name,process-name,series-description,value,units
0,2025-12-12,U.S.,Fuel Ethanol,Exports,U.S. Exports of Fuel Ethanol (Thousand Barrels...,191,MBBL/D
1,2025-12-12,U.S.,Fuel Ethanol,Refinery and Blender Net Input,U.S. Refiner and Blender Net Input of Fuel Eth...,906,MBBL/D
2,2025-12-12,U.S.,Fuel Ethanol,Ending Stocks,U.S. Ending Stocks of Fuel Ethanol (Thousand B...,22353,MBBL
3,2025-12-12,U.S.,Fuel Ethanol,Imports,U.S. Imports of Fuel Ethanol (Thousand Barrels...,0,MBBL/D
4,2025-12-12,U.S.,Fuel Ethanol,Oxygenate Plant Production,U.S. Oxygenate Plant Production of Fuel Ethano...,1131,MBBL/D
5,2025-12-05,U.S.,Fuel Ethanol,Refinery and Blender Net Input,U.S. Refiner and Blender Net Input of Fuel Eth...,851,MBBL/D
6,2025-12-05,U.S.,Fuel Ethanol,Oxygenate Plant Production,U.S. Oxygenate Plant Production of Fuel Ethano...,1105,MBBL/D
7,2025-12-05,U.S.,Fuel Ethanol,Imports,U.S. Imports of Fuel Ethanol (Thousand Barrels...,0,MBBL/D
8,2025-12-05,U.S.,Fuel Ethanol,Ending Stocks,U.S. Ending Stocks of Fuel Ethanol (Thousand B...,22510,MBBL
9,2025-12-05,U.S.,Fuel Ethanol,Exports,U.S. Exports of Fuel Ethanol (Thousand Barrels...,125,MBBL/D
